# /// Accourding to the **Question3 Part(3-3)** 
# /// I used this code and apply changes **(Findding best Learning Rate)**. 

## /// **[06_ConvolutionalNeuralNetwork-Hoda-Keras.ipynb](https://https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/blob/master/homework/nbviewer.jupyter.org/github/Alireza-Akhavan/SRU-deeplearning-workshop/blob/master/06_ConvolutionalNeuralNetwork-Hoda-Keras.ipynb)**

In [ ]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

In [27]:
# Import libraries and modules
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# **Declaring a simple Callback**
# when our **validation reached 75%** the Training proccess will be stopped  

In [28]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.75):
            print("\nReached 75% validation so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [29]:
np.random.seed(123)  # for reproducibility
# Load pre-shuffled HODA data into train and test sets
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size=3500,
                                                                        test_sample_size=400,size=28)

In [30]:
# Preprocess input data
''' 3.1: input data in numpy array format'''
x_train = np.array(x_train_original)
x_test = np.array(x_test_original)
'''3.2 normalize our data values to the range [0, 1]'''
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [31]:
# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [32]:
# Preprocess class labels
y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test = keras.utils.to_categorical(y_test_original, num_classes=10)

In [33]:
# test and validation set
x_val = x_test[:200]
x_test = x_test[200:]
y_val = y_test[:200]
y_test = y_test[200:]

In [34]:
# Define model architecture
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [35]:
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
# Fit model on training data
history = model.fit(x_train, y_train,
          epochs=200, batch_size=256, validation_data = (x_val, y_val), callbacks=[callbacks])

Epoch 1/200
14/14 [==============================] - 3s 206ms/step - loss: 2.1048 - accuracy: 0.3294 - val_loss: 1.4977 - val_accuracy: 0.7450
Epoch 2/200
14/14 [==============================] - ETA: 0s - loss: 1.2133 - accuracy: 0.6126
Reached 75% validation so cancelling training!
14/14 [==============================] - 3s 194ms/step - loss: 1.2133 - accuracy: 0.6126 - val_loss: 0.5736 - val_accuracy: 0.8350


# **Accourding to the result:**
## **in the first epoch:** validation is `0.7450`
## **and in the second epoch:** our validation reaches `0.8350` 
## which is more than 75% 
## so the training proccess is forced to stop by the callback
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////